# Frame design



### Table of contents

1. [Frame Drawing](#section2)
2. [Parameters definition](#section4)
3. [Sizing code](#section5)
4. [DoE for the initial conditions](#section6)


In [1]:
from math import pi, sin, cos, tan
import scipy
import scipy.optimize
from math import pi
from math import sqrt
import math
import timeit


import numpy as np
from pyDOE import *

<a id='#section1'></a>

<a id='#section2'></a>

## Frame drawing

*Example of sketch of the structure of a multi-rotor drone with 4 arms*  
![4-arms drone structure](./img/FrameDesign.jpg)


<a id='#section3'></a>

<a id='#section4'></a>

## Parameters definition

### General specifications

In [2]:
# Input Geometrical dimensions
Larm=0.35 # [m] one arm length
Narm=4 # [-] arms number
VolBat=0.132*0.043*0.027 #[m^3] Volume Battery (https://www.miniplanes.fr/eflite-accu-lipo-4s-148v-3300mah-50c-prise-ec3)


# Specifications for take off
F_to=32 # [N] global drone force for the take off
M_total=2 # [kg] total drone mass

# Specifications for landing impact
v_impact=1 # [m/s] impact speed

#Payload specifications
M_camera=0.113#[g] camera mass
H_camera=0.057#[m] height camera (https://www.flir.de/products/vue-pro/)

### Material assumptions

In [3]:
# Material properties

# for beeam and core
Ey_bc=70.3e9 # [Pa] Young modulus
Rho_bc=2700 # [kg/m^3] Volumic mass
Sigma_bc=80e6 # [Pa] Elastic strength

# for landing gear 
Ey_lg=2e9 # [Pa] Young modulus
Rho_lg=1070 # [kg/m^3] Volumic mass
Sigma_lg=39e6 # [Pa] Elastic strength



### Design assumptions (constant)

In [4]:
k_sec=4 # [-] security coefficient

### Design variable (to optimize)

In [5]:
k_TH=0.1 # [-] aspect ratio : ratio thickness (T) / side of the beam (H) < 1
k_BH=2 # [-] aspect ratio : ratio body height (Hbody)/ height beam (H) > 1
Teta=20/90*pi/2 # [rad] landing gear angle (0 is vertical beam) 0<Teta<90
k_TT=1 # [-] aspect ratio : ratio landing gear thickness (Tlg)/ thickness beam (T). > 1
k_L=0.5 # [-] aspect ratio: Length body(Lbody)/length arm (Larm)<1
Hlg=.1 # [m] Height of landing gear (space for battery or sensors)

#Vector of parameters
parameters= scipy.array((k_TH,k_BH,Teta,k_TT,k_L,Hlg))

<a id='#section5'></a>

## Sizing Code

The set of equations of a sizing code can generate typical issues such : 
- Underconstrained set of equations: the lacking equations can come from additional scenarios, estimation models or additional sizing variable. 
- overconstrained equations often due to the selection of a component on multiple critera: the adding of over-sizing coefficients and constraints in the optimization problem can generally fix this issue 
- algebraic loops often due to selection criteria requiring informations generally available after the selection 

**Over-constraint singularities** Example: two variables in one equation:

- Equation: cross section side of a beam resisting a normal stress: $\displaystyle H=\sqrt[3]{\frac{6*M_{to}}{\sigma_{bc}*(1-(1-2*T)^4)}}$

- Variables: thickness ($T$), cross section side ($H$)

- Geometrical restriction:$\displaystyle T<H$ 

- Strategy: $\displaystyle T=k_{TH}*H$ where 0<$k_{TH}$<1

The equation is thus transformed into an inequality and through a large number of iterations the value of both variables can be estimated.
 $\displaystyle H>\sqrt[3]{\frac{6*M_{to}}{\sigma_{bc}*(1-(1-2*k_{TH})^4)}}$

**Algebraic loop** : Total mass depends on the different geometrical components.



The final optimization problem depends thus of these parameters:

- $k_{TH}$: aspect ratio : ratio thickness (T) / side of the beam (H) < 1
- $k_{BH}$ aspect ratio : ratio body height (Hbody)/ height beam (H) > 1
- $ \theta$ landing gear angle (0 is vertical beam) 0<Teta<90
- $k_{TT}$ ratio landing gear thickness ( body side dimensions)
- $k_{L}$ aspect ratio: Length body(Lbody)/length arm (Larm)
- $Hlg$: Height of landing gear (space for battery or sensors)

The sizing code is defined here in a function which can give:

- an evaluation of the objective: here the frame mass
- an evaluation of the constraints: here the normal stress at the landing gear and body core, battery dimensions.


In [10]:
def SizingCode(param,arg):

    #Variables
    k_TH=param[0]
    k_BH=param[1]
    Teta=param[2]
    k_TT=param[3]
    k_L=param[4]
    Hlg=param[5]
    
    #### Beam Sizing - Take Off

    M_to=F_to/Narm*Larm*k_sec # [N.m] Moment applied in the drone center
    

#    H=(M_to/Sigma_bc/(1-(1-2*k_TH)**4))**(1/3) # [m] Side length of the beam
    H=(6*M_to/Sigma_bc/(1-(1-2*k_TH)**4))**(1/3) # [m] Side length of the beam
    T=k_TH*H # [m] Thickness of the side beam


    #### Body and Landing gear sizing - Landing impact
    # Body stiffness calculation 
    Hbody=k_BH*H # [m] height of the body
    Ibody=1/12*((H+2*T)*Hbody**3-H*(Hbody-2*T)**3) # [m^4] Section inertia of the body
    Lbody=k_L*Larm #[m] length of the body
    K1=3*Ey_bc*Ibody/(Lbody)**3 # [N/m] equivalent stiffness of the body

    # Landing gear stiffness calculation 
    Llg=Hlg/cos(Teta) # [m] Landing gear length
    Tlg=k_TT*T # [m] landing gear thickness
    
    Ilg=1/12*(Tlg**4) # [m^4] Section inertia of the landing gear rectangular section
    K2=3*Ey_lg*Ilg/Llg**3/sin(Teta) # [N/m] equivalent stiffness of the landing gear

    # Global stiffness
    Kg=K1*K2/(K1+K2)*Narm # [N/m] global stiffness of all the arms

    # Impact force
    Fimpact= (v_impact*(Kg*M_total)**(1/2)+M_total*9.81)*k_sec # [N] Total impact force, we assume all the landing gear impact together

    # Stress calculation in the landing gear

    M_LG=Fimpact/Narm*Hlg*tan(Teta) # [N.m] Moment applied in the landing gear
    Sigma_lg_impact=M_LG*(Tlg/2)/Ilg # [Pa] Max stress in the landing gear

    # Stress calculation in the body

    M_Body=(Fimpact/Narm*Lbody+M_LG) # [N.m] Moment applied in the body
    Sigma_body_impact=M_Body*(Hbody/2)/Ibody # [Pa] Max stress in the landing gear

    # Mass calculation
    Mbeams=Narm*Larm*(H**2-(H-2*T)**2)*Rho_bc #[kg] Total beams' mass
    MLG=Narm*Llg*Tlg**2*Rho_lg #[kg] Total landing gears' mass
    Mbody=Narm*(Lbody)*(Hbody*(H+2*T)-(Hbody-2*T)*H)*Rho_bc #[kg] Total body's mass

    Mframe=Mbeams+MLG+Mbody #[kg] total frame mass
    Vbody=(2*Lbody)**2*Hbody
    # Contraintes : stress 

    
    constraints = [(Sigma_bc-Sigma_body_impact)/Sigma_body_impact,(Sigma_lg-Sigma_lg_impact)/Sigma_lg_impact,(Vbody-VolBat)/VolBat,(Hlg-Fimpact/(Narm*Kg)-H_camera)/(Hlg)]
    # Objectif : masse totale 
    if arg=='Obj':
        return Mframe
    elif arg=='Prt':
        print("* Optimisation variables:")
        print("           Aspect ratio for the beam's thickness (T/H), k_TH = %.2f"% k_TH)
        print("           Aspect ratio for the body's height (Hbody/H) k_BH = %.2f"% k_BH)
        print("           Angle of the landing gear w.r.t. the beam Teta= %.2f º"% (Teta/pi*180))
        print("           Aspect ratio for the Landing gear's thickness (Tlg/T) k_TT=  %.2f"% k_TT)
        print("           Aspect ratio: Length body(Lbody)/length arm (Larm) k_L=  %.2f"% k_L)
        print("           Landing gear height, Hlg = %.5f m" % Hlg)
        print("* Components characteristics:")
        print("           Only for Take-Off:" )
        print("               Arm length, Larm = %.5f m" % Larm)        
        print("               Side length cross section of the beam, H = %.5f m" % H)
        print("               Thickness of the beam, T= %.5f m" % T)
        print("           Only for Landing:" )
        print("               Body length, Lbody = %.5f m" % Lbody)
        print("               Body cross section height, Hbody = %.5f m" % Hbody)
        print("               Body cross section width, H+2T = %.5f m" % (H+2*T))
        print("               Landing gear cross section width, Tlg = %.5f m" % Tlg)
        print("               Impact speed, vimpact = %.5f m/s" % v_impact)
        print("               K1 = %.2f N/m" % K1)
        print("               K2 = %.2f N/m" % K2)
#        print("               Ibody = %.5e m^4" % Ibody)
#        print("               Ilg = %.5e m^4" % Ilg)         
#        print("               M_LG = %.5f N/m" % M_LG)
#        print("               M_Body = %.2f N/m" % M_Body)        
        print("               Total stiffness, K= %.5f N/m" % Kg)
        print("               Security coefficient= %.5f" %k_sec)
        print("               Impact force, Fimpact = %.5f N" % Fimpact)       
        print("               Max stress in the landing gear, Sigma_lg_impact = %.5f N" % Sigma_lg_impact)       
        print("               Max stress in the body, Sigma_body_impact = %.5f N/m^2" % Sigma_body_impact)    
        print("               Total beams mass = %.5f kg"%Mbeams)
        print("               Total lg mass = %.5f kg"%MLG)
        print("               Total body mass = %.5f kg"%Mbody)
        
        print("* Normalized Constraints (should be >0):")
        print("           Stress margin at the Body: (Sigma_bc-Sigma_body_impact)/Sigma_body_impact= %.3e" % constraints[0])
        print("           Stress margin at the landing gears: (Sigma_lg-Sigma_lg_impact)/Sigma_lg_impact= %.3e " %constraints[1])
        print("           (Vbody-VolBat)/VolBat= %.3e " %constraints[2])
        print("           (Hlg-Fimpact/(Narm*Kg)-H_camera)/(Hlg)= %.3e " %constraints[3])
    else:
        return constraints

<a id='#section6'></a>

## DoE for the initial values

The choice of an initial set of values can determine the final result. Through this code, we randomize a design of experiments for the initial values vector and we save only the one, which gives us in the end positive contraints. That means that the inequalities are satisfied.

In [11]:
def MultiStart_SLSQP(objectif, xlimits, contraints,iteration,accuracy,samples):
   
#1: DoE for the initial conditions
#------------    
    minimum=xlimits[:,0] #lower limit contraints
    maximum=xlimits[:,1] #upper limit contraints
    diff=maximum-minimum 

    design=lhs(len(minimum), samples) # doe for X-samples distributed along 0 and 1.

    doe = [vec *diff+minimum for vec in design] # doe transformed to our array of initial parameters 

    
    
# 2 : slsqp optimizing loop: find the minimum objective and its parameter vector
#------------    
    
    obj=[] # vector to save objectives which constraints are positive


    for vector in doe: #slsqp algorithm for the random parameters
                
        result = scipy.optimize.fmin_slsqp(func=objectif, x0=vector,bounds=xlimits,
                                           f_ieqcons=contraints, iter=iteration,
                                           acc=accuracy, iprint =-1)

        if all (value>0 for value in contraints(result)): #only consider positive contraints!
            
            obj.append(objectif(result)) #saving objective with positive contraints

            if min(obj)==objectif(result): #objectif is examined if it is the minimum mass from the vector obj
                min_obj=result #choose minimum objective
    return min_obj

In the next code, we implement the function. And very important is that we set truthful limits where our initial values be found.

In [12]:
import numpy as np

smpl=50 #samples
lims = np.array([
[0.15,0.4],# k_TH [-] aspect ratio : ratio thickness (T) / side of the beam (H) < 1 
[1,4],# k_BH [-] aspect ratio : ratio body height (Hbody)/ height beam (H) > 1
[30/90*pi/2,pi/2],# Teta [rad] landing gear angle (0 is vertical beam) 0<Teta<90
[1,100], #k_TT aspect ratio : ratio landing gear thickness (Tlg)/ height beam (T). > 1
[0,1], # k_L aspect ratio: Length body(Lbody)/length arm (Larm)<1
[0.01,1.165], # H_lG [m] High of landing gear (space for battery or sensors)
]) # ranges of contraints
    
iter=1500
acc=1e-8

# optimization with SLSQP algorithm
contrainte=lambda x: SizingCode(x, 'Const') # calling functions from the SizingCode
objectif=lambda x: SizingCode(x, 'Obj') # calling functions from the SizingCode

min_vec=MultiStart_SLSQP(objectif,lims,contrainte,iter,acc, smpl)


C:\Users\Aitor\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: RuntimeWarning: divide by zero encountered in double_scalars
C:\Users\Aitor\Anaconda3\lib\site-packages\ipykernel_launcher.py:36: RuntimeWarning: invalid value encountered in double_scalars


In [13]:
print("-----------------------------------------------")
print("Final characteristics after optimization :")

SizingCode(min_vec, 'Prt')

print("-----------------------------------------------")

-----------------------------------------------
Final characteristics after optimization :
* Optimisation variables:
           Aspect ratio for the beam's thickness (T/H), k_TH = 0.15
           Aspect ratio for the body's height (Hbody/H) k_BH = 3.80
           Angle of the landing gear w.r.t. the beam Teta= 30.00 º
           Aspect ratio for the Landing gear's thickness (Tlg/T) k_TT=  14.13
           Aspect ratio: Length body(Lbody)/length arm (Larm) k_L=  0.09
           Landing gear height, Hlg = 0.08779 m
* Components characteristics:
           Only for Take-Off:
               Arm length, Larm = 0.35000 m
               Side length cross section of the beam, H = 0.01034 m
               Thickness of the beam, T= 0.00155 m
           Only for Landing:
               Body length, Lbody = 0.03121 m
               Body cross section height, Hbody = 0.03933 m
               Body cross section width, H+2T = 0.01344 m
               Landing gear cross section width, Tlg = 0.02192 m
